# Chapter 16: Advanced Index Types

While B-tree indexes handle most OLTP workloads, specialized data types and query patterns require alternative index structures. This chapter covers PostgreSQL's advanced index types—Hash, GIN, GiST, BRIN, and SP-GiST—detailing their architectures, optimal use cases, and operational considerations for production environments.

## 16.1 Hash Indexes

Hash indexes store hash codes of values rather than the values themselves, enabling fast equality lookups with minimal storage overhead.

### 16.1.1 Architecture and Recovery

```sql
-- Create hash index (syntax only, use cases discussed below)
CREATE INDEX idx_users_email_hash ON users USING HASH (email);

-- Hash index structure:
-- 1. Hash function converts column value to 32-bit hash code
-- 2. Buckets store hash codes pointing to heap tuples
-- 3. Collision handling: Multiple values may hash to same bucket (chaining)

-- Critical historical note:
-- Before PostgreSQL 10: Hash indexes were not WAL-logged
-- - Not replicated to standbys
-- - Lost on crash recovery (needed manual rebuild)
-- PostgreSQL 10+: Full WAL support, replication safe

-- Current limitations (all versions):
-- 1. Only supports equality operators (=)
-- 2. No support for ORDER BY, range queries, or pattern matching
-- 3. No multi-column indexes
-- 4. No unique constraints
-- 5. No covering indexes (INCLUDE clause)
```

### 16.1.2 When to Use Hash Indexes

```sql
-- Legitimate use case: Huge tables with equality-only lookups
-- where B-tree index size becomes prohibitive

-- Comparison: B-tree vs Hash for 100M row table with UUIDs
CREATE INDEX idx_uuid_btree ON events(event_id);  -- B-tree
CREATE INDEX idx_uuid_hash ON events USING HASH (event_id);  -- Hash

-- Hash index advantages:
-- 1. Smaller size (typically 30-40% smaller than B-tree for same data)
-- 2. Faster equality lookups (no tree traversal, direct bucket access)
-- 3. No ordering overhead (B-tree maintains sort order, hash doesn't)

-- Practical example: Event sourcing or write-heavy logging
CREATE TABLE events (
    event_id UUID PRIMARY KEY,  -- B-tree for PK constraint
    aggregate_id UUID NOT NULL,
    event_type TEXT,
    payload JSONB,
    created_at TIMESTAMPTZ DEFAULT NOW()
);

-- Secondary lookup by aggregate_id (equality only):
CREATE INDEX idx_events_agg_hash ON events USING HASH (aggregate_id);
-- Supports: WHERE aggregate_id = 'uuid-value'
-- Does NOT support: WHERE aggregate_id > 'x' (range not supported)

-- Industry standard: Rarely used in modern PostgreSQL
-- B-tree equality performance is nearly as fast as hash
-- B-tree offers more flexibility (range queries, sorting)
-- Only use hash when:
-- 1. Storage is absolutely critical (petabyte-scale)
-- 2. Only equality lookups ever needed (guaranteed)
-- 3. Index size causing cache pressure
```

### 16.1.3 Hash Index Maintenance

```sql
-- Hash indexes can degrade with high collision rates
-- Monitor with:
SELECT
    indexrelname,
    pg_size_pretty(pg_relation_size(indexrelid)) as size,
    idx_scan,
    idx_tup_read
FROM pg_stat_user_indexes
WHERE indexrelname LIKE '%hash%';

-- Rebuilding hash indexes (if performance degrades):
REINDEX INDEX CONCURRENTLY idx_events_agg_hash;

-- Hash index sizing:
-- Default fillfactor: 90 (like B-tree)
-- Lower fillfactor reduces collisions but increases size:
CREATE INDEX idx_hash_ff ON events USING HASH (aggregate_id) WITH (fillfactor = 70);
```

## 16.2 GIN (Generalized Inverted Index)

GIN indexes are optimized for data types where a single item contains multiple values (arrays, JSONB, full-text search documents). They excel at containment queries.

### 16.2.1 GIN Architecture

```sql
-- GIN stores inverted index: value -> list of row locations
-- Structure:
-- Entry tree (B-tree of values) -> Posting lists (row locations)
-- 'error' -> [(0,1), (0,5), (1,3)...]  -- All rows containing 'error'
-- 'warning' -> [(0,2), (0,7)...]

-- Best for:
-- 1. Arrays: @> (contains), && (overlap)
-- 2. JSONB: @> (containment), ? (key exists)
-- 3. Full text search: @@ (tsquery match)
-- 4. Range types: && (overlap), @>, <@

-- Create GIN index for JSONB
CREATE INDEX idx_logs_payload ON logs USING GIN (payload);

-- Query using GIN index:
SELECT * FROM logs 
WHERE payload @> '{"service": "api", "level": "error"}';
-- GIN index finds rows where payload contains these key-value pairs
```

### 16.2.2 JSONB Indexing Strategies

```sql
-- Default GIN index (jsonb_ops): Supports all JSONB operators
CREATE INDEX idx_data_gin ON data USING GIN (json_column);

-- Supports:
-- @> (JSON containment): WHERE json_column @> '{"a":1}'
-- ? (key exists): WHERE json_column ? 'key'
-- ?| (any key exists): WHERE json_column ?| array['a','b']
-- ?& (all keys exist): WHERE json_column ?& array['a','b']

-- Optimized GIN index (jsonb_path_ops): Smaller, faster containment
CREATE INDEX idx_data_path ON data USING GIN (json_column jsonb_path_ops);
-- Only supports @> (containment), but 30-50% smaller and faster for @>
-- Does NOT support: ?, ?|, ?& (key existence)

-- Decision matrix:
-- If queries use only @> (containment): Use jsonb_path_ops
-- If queries use key existence (?, ?|): Use default jsonb_ops
-- If mixed: Create both indexes (rare) or use jsonb_ops

-- Practical example: Document store with tags
CREATE TABLE articles (
    id SERIAL PRIMARY KEY,
    title TEXT,
    content JSONB,
    tags TEXT[]
);

-- GIN for JSONB content:
CREATE INDEX idx_articles_content ON articles USING GIN (content);

-- Query:
SELECT * FROM articles 
WHERE content @> '{"author": "Alice", "published": true}';

-- GIN for array tags (separate index):
CREATE INDEX idx_articles_tags ON articles USING GIN (tags);
-- Supports: WHERE tags && ARRAY['postgres', 'database']  -- Overlap
-- Supports: WHERE tags @> ARRAY['postgres']  -- Contains
```

### 16.2.3 Full-Text Search with GIN

```sql
-- Full-text search requires tsvector (document) and tsquery (search terms)
-- GIN is the standard index for full-text search

-- Step 1: Create tsvector column (or use expression index)
CREATE TABLE documents (
    id SERIAL PRIMARY KEY,
    title TEXT,
    body TEXT,
    search_vector tsvector  -- Generated column
);

-- Step 2: Populate tsvector (typically via trigger)
UPDATE documents SET search_vector = 
    setweight(to_tsvector('english', coalesce(title, '')), 'A') ||
    setweight(to_tsvector('english', coalesce(body, '')), 'B');

-- Step 3: Create GIN index
CREATE INDEX idx_fts_gin ON documents USING GIN (search_vector);

-- Query (uses index):
SELECT * FROM documents 
WHERE search_vector @@ to_tsquery('english', 'postgres & (indexing | search)');
-- & = AND, | = OR, ! = NOT, <-> = phrase

-- Alternative: Expression index (no separate column)
CREATE INDEX idx_fts_expr ON documents 
USING GIN (to_tsvector('english', title || ' ' || body));
-- Query must match exactly:
SELECT * FROM documents 
WHERE to_tsvector('english', title || ' ' || body) @@ to_tsquery('postgres');
```

### 16.2.4 GIN Fast Update and Pending List

```sql
-- GIN indexes have "fastupdate" feature (on by default)
-- Inserts go to pending list first, then bulk merged into main index later
-- Reduces random I/O for writes, but slows reads if pending list large

-- Check pending list size:
SELECT
    indexrelname,
    pg_size_pretty(pg_relation_size(indexrelid)) as index_size,
    (pg_stat_user_indexes.idx_tup_read + pg_stat_user_indexes.idx_tup_fetch) as usage
FROM pg_stat_user_indexes
WHERE indexrelname = 'idx_logs_payload';

-- If pending list grows large (millions of entries), queries slow down
-- Force immediate cleanup:
VACUUM ANALYZE logs;  -- Merges pending list into main index

-- Disable fastupdate (if consistent write performance needed):
CREATE INDEX idx_logs_payload_no_pending ON logs 
USING GIN (payload) WITH (fastupdate = off);
-- Writes slower (immediate index update), but reads always fast
-- Alternative: Keep fastupdate on, schedule frequent vacuums

-- GIN index size considerations:
-- GIN indexes can be large (especially for text-heavy JSONB)
-- jsonb_path_ops creates smaller indexes than jsonb_ops
-- Consider partial GIN indexes for hot subsets:
CREATE INDEX idx_logs_errors ON logs USING GIN (payload) 
WHERE payload @> '{"level": "error"}';
-- Much smaller than indexing all logs
```

## 16.3 GiST (Generalized Search Tree)

GiST indexes support "nearest-neighbor" searches and are ideal for geometric data, ranges, and trigram matching. They support lossy indexing (recheck required).

### 16.3.1 GiST Architecture and Flexibility

```sql
-- GiST is a balanced tree framework for non-scalar data
-- Supports arbitrary predicates (not just equality/range)
-- Common uses:
-- 1. Geometric types (point, box, circle): nearest neighbor
-- 2. Range types (int4range, tstzrange): overlap/containment
-- 3. Text: trigram similarity (fuzzy search)
-- 4. Network types (inet, cidr): network containment

-- Range type example:
CREATE TABLE reservations (
    id SERIAL PRIMARY KEY,
    room_id INTEGER,
    during TSTZRANGE  -- Timestamp range
);

-- GiST index for range overlap queries:
CREATE INDEX idx_reservations_during ON reservations USING GIST (during);

-- Query: Find conflicting reservations (overlap)
SELECT * FROM reservations 
WHERE during && '[2024-06-01 14:00, 2024-06-01 16:00]'::tstzrange;
-- && = overlaps operator
-- GiST efficiently finds ranges that overlap with query range

-- Nearest neighbor (KNN) with GiST:
CREATE TABLE locations (
    id SERIAL PRIMARY KEY,
    name TEXT,
    geo POINT  -- (x,y) or (lon,lat)
);

CREATE INDEX idx_locations_geo ON locations USING GIST (geo);

-- Find 5 nearest locations to point (10, 20):
SELECT * FROM locations 
ORDER BY geo <-> point(10, 20) 
LIMIT 5;
-- <-> = distance operator
-- GiST supports KNN searches (B-tree cannot do this efficiently)
```

### 16.3.2 Trigram Indexes for Fuzzy Text Search

```sql
-- Trigram (pg_trgm) extension for pattern matching
CREATE EXTENSION IF NOT EXISTS pg_trgm;

-- GiST trigram index (smaller, faster for similarity, slower for exact):
CREATE INDEX idx_users_name_trgm ON users USING GIST (name gist_trgm_ops);

-- GIN trigram index (larger, faster for exact matches):
CREATE INDEX idx_users_name_trgm_gin ON users USING GIN (name gin_trgm_ops);

-- Use cases:
-- 1. Typo-tolerant search (similarity)
-- 2. Leading wildcard LIKE: '%smith'
-- 3. Regex matching: ~ '.*john.*'

-- Similarity search (GiST preferred):
SELECT * FROM users 
WHERE name % 'Jon Smith';  -- % = similarity operator
-- Returns rows with similar trigram composition (e.g., "John Smith", "Jon Smyth")
-- Similarity threshold default: 0.3 (30% matching trigrams)

-- Leading wildcard (both GIN and GiST work):
SELECT * FROM users WHERE name LIKE '%smith';
-- Cannot use B-tree (requires prefix), but trigram index handles this

-- Regex:
SELECT * FROM users WHERE name ~* '^j.*smith$';
-- Uses trigram index for prefix 'j' and suffix 'smith'

-- Performance note:
-- GiST trigram indexes are lossy (recheck filter applied)
-- GIN trigram indexes are exact (no recheck needed for equality)
-- For high-selectivity queries, GIN is faster; for low-selectivity, GiST is smaller
```

### 16.3.3 Geometric and Nearest-Neighbor Search

```sql
-- PostGIS (spatial extension) uses GiST internally, but native geometric types:
CREATE TABLE places (
    id SERIAL PRIMARY KEY,
    name TEXT,
    location GEOGRAPHY(POINT, 4326)  -- PostGIS, but concept similar
);

-- Native point type (no PostGIS required):
CREATE TABLE points (
    id SERIAL PRIMARY KEY,
    coord POINT
);

CREATE INDEX idx_points_coord ON points USING GIST (coord);

-- Bounding box queries:
SELECT * FROM points 
WHERE coord <@ box '((0,0),(10,10))';  -- Contained in box

-- Distance queries:
SELECT *, coord <-> point(5,5) as dist 
FROM points 
ORDER BY coord <-> point(5,5) 
LIMIT 10;
-- Uses GiST index for ordered distance scan
```

## 16.4 BRIN (Block Range INdex)

BRIN indexes store summary statistics about block ranges (groups of pages), making them tiny and efficient for very large, naturally ordered tables.

### 16.4.1 BRIN Architecture

```sql
-- BRIN structure:
-- Table divided into block ranges (default 128 pages per range)
-- Stores min/max value for each range
-- Query checks summary: If query value outside min/max, skip range

-- Ideal for:
-- 1. Time-series data (timestamps naturally increasing)
-- 2. Sensor data (ordered by time)
-- 3. Any large table with strong correlation between physical order and value

-- Create BRIN index:
CREATE TABLE sensor_readings (
    id BIGSERIAL,
    sensor_id INTEGER,
    measured_at TIMESTAMPTZ,
    value NUMERIC
);

-- BRIN index (tiny, fast to create):
CREATE INDEX idx_sensor_time_brin ON sensor_readings 
USING BRIN (measured_at);

-- Size comparison for 1 billion rows:
-- B-tree: ~20 GB
-- BRIN: ~1 MB (stores only min/max per 128-page range)

-- Query using BRIN:
SELECT * FROM sensor_readings 
WHERE measured_at > NOW() - INTERVAL '1 hour';
-- Scans BRIN summary: "Which ranges have max > (NOW() - 1 hour)?"
-- Scans only those specific heap blocks
```

### 16.4.2 Correlation and BRIN Effectiveness

```sql
-- BRIN effectiveness depends on correlation:
-- Correlation = 1.0: Perfect ordering (ideal for BRIN)
-- Correlation = 0.0: Random order (BRIN useless, scans everything)

-- Check correlation:
SELECT attname, correlation 
FROM pg_stats 
WHERE tablename = 'sensor_readings' AND attname = 'measured_at';
-- If correlation < 0.9, BRIN may not be effective

-- Maintaining correlation:
-- 1. Use time-series partitioning (declarative partitioning by range)
-- 2. CLUSTER table on timestamp (one-time sort, expensive lock)
-- 3. Insert in chronological order (append-only)

-- BRIN with pages_per_range tuning:
CREATE INDEX idx_sensor_time_brin_dense ON sensor_readings 
USING BRIN (measured_at) WITH (pages_per_range = 32);
-- Smaller ranges = more precise but larger index
-- Default 128 pages (~1MB range), 32 pages = 256KB range
-- Use smaller ranges if data has some out-of-order insertion

-- BRIN limitations:
-- 1. Only works for range queries (>, <, BETWEEN), not equality
-- 2. Requires high correlation to be effective
-- 3. Always bitmap scan (never index-only or direct tuple access)
-- 4. No support for multi-column (but can create multiple BRIN indexes)
```

### 16.4.3 BRIN for Non-Time-Series Data

```sql
-- BRIN can work for any naturally ordered data:
CREATE TABLE events (
    event_id BIGSERIAL,
    account_id INTEGER,  -- Naturally increasing as accounts created
    event_type TEXT,
    created_at TIMESTAMPTZ
);

-- If account_id is roughly increasing with time (correlation > 0.9):
CREATE INDEX idx_events_account_brin ON events USING BRIN (account_id);

-- Query recent accounts:
SELECT * FROM events WHERE account_id BETWEEN 1000000 AND 1100000;
-- Effective if recent accounts are in recent heap blocks

-- BRIN on multiple columns (separate indexes):
CREATE INDEX idx_events_time_brin ON events USING BRIN (created_at);
-- PostgreSQL can combine multiple BRIN bitmaps with BitmapAnd
```

## 16.5 SP-GiST (Space-Partitioned GiST)

SP-GiST is designed for data with natural clustering hierarchies (prefix trees, quadtrees, k-d trees). It excels for text prefixes and IP addresses.

### 16.5.1 SP-GiST Architecture

```sql
-- SP-GiST uses space-partitioning trees:
-- - Text: Trie (prefix tree)
-- - Points: Quadtree or k-d tree
-- - Networks: Patricia trie for IP addresses

-- Common use cases:
-- 1. Text prefix searches (like 'abc%')
-- 2. IP address ranges (inet/cidr)
-- 3. Point data in 2D space

-- Text prefix example (alternative to B-tree for long prefixes):
CREATE TABLE phone_directory (
    id SERIAL PRIMARY KEY,
    phone_number TEXT,  -- E.g., '+1-555-123-4567'
    name TEXT
);

-- SP-GiST for prefix searches:
CREATE INDEX idx_phone_prefix ON phone_directory 
USING SPGIST (phone_number);

-- Query:
SELECT * FROM phone_directory 
WHERE phone_number LIKE '+1-555-123%';
-- SP-GiST traverses the trie down to '+1-555-123' then scans leaves
-- More efficient than B-tree for long common prefixes
```

### 16.5.2 Network Address Indexing

```sql
-- IP address containment with SP-GiST:
CREATE TABLE ip_ranges (
    id SERIAL PRIMARY KEY,
    network CIDR,
    description TEXT
);

CREATE INDEX idx_ip_network ON ip_ranges USING SPGIST (network);

-- Query: Find network containing specific IP:
SELECT * FROM ip_ranges 
WHERE network >> '192.168.1.100'::inet;
-- >> = contains operator
-- SP-GiST efficiently finds containing networks using Patricia trie

-- Alternative: GiST also supports network types, but SP-GiST is often smaller/faster
-- for strict containment hierarchies
```

## 16.6 Partial and Expression Indexes (Advanced Patterns)

While introduced in Chapter 15, advanced index types enable sophisticated partial and expression strategies.

### 16.6.1 Partial Indexes with Advanced Types

```sql
-- Partial GIN index (hot data only):
CREATE INDEX idx_recent_errors ON logs 
USING GIN (payload) 
WHERE payload @> '{"level": "error"}' 
  AND created_at > NOW() - INTERVAL '7 days';
-- Only indexes recent errors, keeping index small and fast

-- Partial GiST for active reservations:
CREATE INDEX idx_active_reservations ON reservations 
USING GIST (during) 
WHERE status = 'active';
-- Only index active bookings (past bookings archived but not indexed)

-- BRIN with partial predicate (PostgreSQL 14+):
CREATE INDEX idx_recent_sensor ON sensor_readings 
USING BRIN (measured_at) 
WHERE sensor_id = 1;
-- Per-sensor BRIN indexes for time-series queries
```

### 16.6.2 Expression Indexes with Advanced Types

```sql
-- Functional GIN index for JSONB extraction:
CREATE INDEX idx_json_extract ON events 
USING GIN ((payload -> 'metadata' -> 'tags'));
-- Index specific JSONB path for fast containment queries on that sub-object

-- Expression index with text search:
CREATE INDEX idx_fts_title ON documents 
USING GIN (to_tsvector('english', title));
-- Full-text search on specific column only

-- Range expression index:
CREATE INDEX idx_during_tz ON events 
USING GIST (TSTZRANGE(created_at, created_at + duration, '[)'));
-- Create range on-the-fly from separate columns
```

## 16.7 Index Selection Decision Matrix

```sql
-- Decision tree for index type selection:

-- 1. Equality only on scalar data?
--    -> B-tree (standard) or Hash (rare, space-constrained)

-- 2. Range queries (>, <, BETWEEN) on scalar?
--    -> B-tree (standard) or BRIN (if huge table + high correlation)

-- 3. JSONB containment (@>) or array overlap (&&)?
--    -> GIN (jsonb_ops or jsonb_path_ops)

-- 4. Range type operations (&&, @>) or KNN searches?
--    -> GiST

-- 5. Fuzzy text matching (%, LIKE '%suffix')?
--    -> GiST (trigram) or GIN (trigram)

-- 6. Network containment (IP addresses)?
--    -> SP-GiST (inet/cidr)

-- 7. Text prefix matching (LIKE 'prefix%')?
--    -> B-tree (standard) or SP-GiST (long common prefixes)

-- 8. Full-text search (@@)?
--    -> GIN (tsvector)

-- 9. Geospatial (nearest neighbor)?
--    -> GiST (point/PostGIS)
```

## 16.8 Maintenance and Monitoring

### 16.8.1 Advanced Index Maintenance

```sql
-- GIN index bloat check:
SELECT
    indexrelname,
    pg_size_pretty(pg_relation_size(indexrelid)) as size,
    idx_scan
FROM pg_stat_user_indexes
WHERE indexrelname LIKE 'idx_%gin';

-- GiST index reindexing (if lossy scans increase):
REINDEX INDEX CONCURRENTLY idx_reservations_during;

-- BRIN index refresh (summaries may become stale):
-- BRIN indexes don't bloat like B-trees, but summaries become inaccurate
-- if heap order degrades. Check with:
SELECT
    indexrelname,
    pg_size_pretty(pg_relation_size(indexrelid)) as size,
    idx_scan,
    idx_tup_read,
    idx_tup_fetch
FROM pg_stat_user_indexes
WHERE indexrelname LIKE 'idx_%brin';
-- If idx_tup_fetch >> idx_tup_read, BRIN is scanning many false positives

-- Fix BRIN accuracy:
-- 1. VACUUM (updates visibility map)
-- 2. Rebuild if correlation degraded: REINDEX INDEX CONCURRENTLY idx_brin
-- 3. Or use BRIN with smaller pages_per_range for better precision
```

### 16.8.2 Performance Anti-Patterns

```sql
-- Anti-pattern 1: GIN on low-cardinality JSONB
-- Bad: Indexing JSONB with only 10 distinct templates
-- GIN becomes large, queries return many rows (slow bitmap scans)
-- Solution: B-tree on expression or partial indexes for specific patterns

-- Anti-pattern 2: GiST for high-cardinality exact matches
-- Bad: Using GiST trigram for UUID exact matches
-- GiST is lossy, requires recheck; GIN or B-tree better for exact equality

-- Anti-pattern 3: BRIN on randomly inserted data
-- Bad: BRIN on user_id where IDs are random UUIDs
-- Correlation ~0, BRIN scans almost entire table
-- Solution: B-tree or hash

-- Anti-pattern 4: Multiple expensive indexes on write-heavy tables
-- Bad: GIN + GiST + B-tree on same JSONB column
-- Each index slows writes significantly
-- Solution: Choose one based on query pattern, or use separate tables (CQRS pattern)

-- Anti-pattern 5: Large GIN indexes without fastupdate tuning
-- Bad: 100M row table with GIN, default fastupdate, vacuum runs weekly
-- Pending list grows to millions of entries, queries slow down
-- Solution: Vacuum more frequently, or disable fastupdate if writes are bursty
```

---

## Chapter Summary

In this chapter, you learned:

1. **Hash Indexes**: Store 32-bit hash codes for equality lookups only. Smaller than B-tree but limited to `=` operations. Replicated safely since PostgreSQL 10, but rarely necessary as B-tree equality performance is comparable.

2. **GIN (Generalized Inverted Index)**: Inverted index structure ideal for containment queries. Use for JSONB (`@>`), arrays (`&&`, `@>`), and full-text search (`@@`). Supports `jsonb_path_ops` for smaller, faster containment indexes (but no key existence operators). Uses pending list for fast updates (configurable via `fastupdate` storage parameter).

3. **GiST (Generalized Search Tree)**: Balanced tree framework for non-scalar data. Essential for range types (`&&` overlap), nearest-neighbor searches (`<->` distance operator), and trigram fuzzy matching. Lossy index requiring heap rechecks for some operators.

4. **BRIN (Block Range INdex)**: Stores min/max summaries for page ranges. Extremely small (MBs vs GBs for B-tree) and fast to create. Requires high correlation (>0.9) between physical storage and indexed value (natural for time-series). Ideal for append-only time-series data on `TIMESTAMPTZ` columns.

5. **SP-GiST (Space-Partitioned GiST)**: Partitioned tree structure for hierarchical data. Optimal for text prefix matching (trie structure) and network addresses (Patricia trie for IP containment). Lower memory footprint than GiST for specific access patterns.

6. **Operational Considerations**: GIN indexes require monitoring of pending lists (vacuum frequency). GiST indexes are lossy (recheck overhead). BRIN requires correlation maintenance (avoid random inserts). All advanced indexes have specific query operator requirements—ensure queries match index capabilities.

**Next:** In Chapter 17, we will explore EXPLAIN Like You Mean It—covering detailed plan interpretation, buffer analysis, timing breakdowns, and methodologies for diagnosing plan instability and performance regression.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='15. index_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='17. explain_like_you_mean_it.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
